In [1]:
!pip install pytorch-crf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from crf import load_data, make_labels2i

train_filepath = "./NER_TRAIN_PREAMBLE.json"
dev_filepath = "./NER_DEV_PREAMBLE.json"
train_sents, train_tag_sents = load_data(train_filepath)
dev_sents, dev_tag_sents = load_data(dev_filepath)
labels2i = make_labels2i()

print(len(train_sents))
print()
print("labels2i", labels2i)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "High Court Of Judicature At Allahabad
 
 

       ..." with entities "[(0, 37, 'COURT'), (315, 354, 'PETITIONER'), (371,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "In The High Court For The State Of Telangana
     ..." with entities "[(7, 45, 'COURT'), (190, 201, 'JUDGE'), (330, 348,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during traini

1560

labels2i {'<PAD>': 0, 'B-PRECEDENT': 1, 'B-RESPONDENT': 2, 'B-COURT': 3, 'B-PETITIONER': 4, 'B-PROVISION': 5, 'B-LAWYER': 6, 'B-STATUTE': 7, 'B-CASE_NUMBER': 8, 'B-DATE': 9, 'B-OTHER_PERSON': 10, 'B-JUDGE': 11, 'B-ORG': 12, 'B-GPE': 13, 'B-WITNESS': 14, 'I-PRECEDENT': 15, 'I-RESPONDENT': 16, 'I-COURT': 17, 'I-PETITIONER': 18, 'I-PROVISION': 19, 'I-LAWYER': 20, 'I-STATUTE': 21, 'I-CASE_NUMBER': 22, 'I-DATE': 23, 'I-OTHER_PERSON': 24, 'I-JUDGE': 25, 'I-ORG': 26, 'I-GPE': 27, 'I-WITNESS': 28, 'O': 29}


/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "High Court Of Judicature At Allahabad, Lucknow Ben..." with entities "[(0, 52, 'COURT'), (175, 184, 'PETITIONER'), (213,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [3]:
# Reference assignment 3 : https://github.com/csci5832-f22/assignment_3
from typing import List
def make_features(text: List[str], sent_tags) -> List[List[int]]:
    feature_lists = []
    for i, token in enumerate(text):
        feats = []
        if i > 0:
          prev_word = text[i-1]
          prev_pos  = sent_tags[i-1]
        else:
          prev_word = '<s>'
          prev_pos  = "<s>"
        if i < len(text)-1:
          next_word = text[i+1]
          next_pos  = sent_tags[i+1]
        else:
          next_word = '<s>'
          next_pos  = '<s>'
        feats.append(f"word={token}")
        feats.append(f"pos={sent_tags[i]}")
        feats.append(f"prev_word={prev_word}")
        feats.append(f"prev_pos={prev_pos}")
        feats.append(f"next_word={next_word}")
        feats.append(f"next_pos={next_pos}")
        
        feature_lists.append(feats)
    return feature_lists

In [4]:
import spacy
def featurize(sents: List[List[str]]) -> List[List[List[str]]]:
    nlp = spacy.load("en_core_web_sm")
    feats = []
    for sent in sents:
        sent_tags = []
        docs = [nlp(word) for word in sent]
        for doc in docs:
          for token in doc:
            sent_tags.append(token.pos_)
        feats.append(make_features(sent, sent_tags))

    return feats

In [5]:
import torch
from crf import f1_score, predict, PAD_SYMBOL, pad_features, pad_labels
from tqdm.autonotebook import tqdm
import random

def training_loop(
    num_epochs,
    batch_size,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
    labels2i,
    pad_feature_idx
):
    samples = list(zip(train_features, train_labels))
    
    random.shuffle(samples)
    
    batches = []
    for i in range(0, len(samples), batch_size):
        batches.append(samples[i:i+batch_size])
    
    print("Training...")
    for i in range(num_epochs):
        losses = []
        for batch in tqdm(batches):
            features, labels = zip(*batch)
            features = pad_features(features, pad_feature_idx)
            features = torch.stack(features)
            labels = pad_labels(labels, labels2i[PAD_SYMBOL])
            labels = torch.stack(labels)
            mask = (labels != labels2i[PAD_SYMBOL])

            optimizer.zero_grad()

            log_likelihood = model(features, labels, mask=mask)
            
            negative_log_likelihood = -log_likelihood
            negative_log_likelihood.backward()

            optimizer.step()

            losses.append(negative_log_likelihood.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")

        dev_f1 = f1_score(predict(model, dev_features), dev_labels, labels2i['O'])
        print(f"Dev F1 {dev_f1}")
        
    return model

In [6]:
from crf import build_features_set
from crf import make_features_dict
from crf import encode_features, encode_labels
from crf import NERTagger

train_features = featurize(train_sents)
dev_features = featurize(dev_sents)
all_features = build_features_set(train_features)
features_dict = make_features_dict(all_features)
model = NERTagger(len(features_dict), len(labels2i))

encoded_train_features = encode_features(train_features, features_dict)
encoded_dev_features = encode_features(dev_features, features_dict)
encoded_train_labels = encode_labels(train_tag_sents, labels2i)
encoded_dev_labels = encode_labels(dev_tag_sents, labels2i)

Building features set!


100%|██████████| 1560/1560 [00:00<00:00, 4107.06it/s]


Found 76260 features


In [7]:
num_epochs = 30
batch_size = 16
LR=0.05
optimizer = torch.optim.SGD(model.parameters(), LR)

model = training_loop(
    num_epochs,
    batch_size,
    encoded_train_features,
    encoded_train_labels,
    encoded_dev_features,
    encoded_dev_labels,
    optimizer,
    model,
    labels2i,
    features_dict[PAD_SYMBOL]
)

Training...


  0%|          | 0/98 [00:00<?, ?it/s]

epoch 0, loss: 126.45675990046287


/usr/local/lib/python3.8/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Dev F1 tensor([0.3504])


...

epoch 27, loss: 22.60916212626866
Dev F1 tensor([0.5861])


  0%|          | 0/98 [00:00<?, ?it/s]

epoch 28, loss: 21.864125349083725
Dev F1 tensor([0.5861])


  0%|          | 0/98 [00:00<?, ?it/s]

epoch 29, loss: 21.86725328406509
Dev F1 tensor([0.5857])
